In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix

from xgboost import XGBRegressor

# 한글
import warnings
warnings.filterwarnings('ignore')
mpl.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='Malgun Gothic')





In [2]:
df = pd.read_csv("final.csv", index_col = 0, encoding = "utf-8")


In [3]:
# pd.set_option('display.max_row',100)
# pd.set_option('display.max_column', 100)

In [4]:
df.drop(["시군구", "year_quarter", "addr_road", \
         "transaction_year_month", "transaction_date", \
         "서울_운영_영업_개월_평균", "서울_폐업_영업_개월_평균"], \
        axis = 1, inplace = True)
df["transaction_year"] = df["date"].str.split("-").str[0].astype(int)
df["transaction_month"] = df["date"].str.split("-").str[1].astype(int)
df["transaction_day"] = df["date"].str.split("-").str[2].astype(int)
df.drop("date", axis = 1, inplace = True)

encoder = LabelEncoder()
df["dong_encoded"] = encoder.fit_transform(df["dong"])
df["apt_encoded"] = encoder.fit_transform(df["apt"])
df.drop(["dong", "apt"], axis = 1, inplace = True)

df.rename(columns = {
    "exclusive_use_area": "전용면적",
    "year_of_completion": "건축년도",
    "floor": "층",
    "transaction_real_price": "실거래가",
    "transaction_year": "계약년도",
    "transaction_month": "계약월",
    "transaction_day": "계약일",
    "dong_encoded": "법정동",
    "apt_encoded": "아파트명"
}, inplace = True)
columns = ['소비자태도', '현재생활형편', '미래생활형편', '현재경기판단', '미래경기판단', '내구재구입태도', '주택구입태도', '현재소비지출', '미래소비지출', '순자산', '고용상황', '물가예상']
for column in columns:
    df.rename(columns = {
        column: column + "지수"
    }, inplace = True)
df.rename(columns = {
    "주택매매가격지수": "주택매매가격지수 전국"
}, inplace = True)
df.rename(columns = {
    "아파트실거래가격지수_서울": "아파트실거래가격지수",
    "아파트실거래가격지수_전국": "아파트실거래가격지수 전국",
    "주택매매가격지수_서울_아파트": "주택매매가격지수"
}, inplace = True)
df.rename(columns = {
    "시장금리(콜)": "시장금리"
}, inplace = True)
df.reset_index(inplace = True, drop = True)

In [5]:
df

,전용면적,건축년도,층,실거래가,소비자태도지수,현재생활형편지수,미래생활형편지수,현재경기판단지수,미래경기판단지수,내구재구입태도지수,...,한강 근접 여부,준공 이래 경과년수,초등학교 개수,중학교 개수,고등학교 개수,계약년도,계약월,계약일,법정동,아파트명
0,77.75,1988,5,60000,90.6,81.0,94.9,55.6,80.0,88.0,...,0,27,6,3,4,2015,1,11,8,604
1,77.75,1988,5,65000,90.6,81.0,94.9,55.6,80.0,88.0,...,0,27,6,3,4,2015,3,3,8,604
2,77.75,1988,2,62500,90.6,81.0,94.9,55.6,80.0,88.0,...,0,27,6,3,4,2015,3,7,8,604
3,77.75,1988,6,68000,95.7,86.2,100.4,65.2,83.1,89.3,...,0,27,6,3,4,2015,8,16,8,604
4,79.97,1987,5,73000,90.6,81.0,94.9,55.6,80.0,88.0,...,0,28,6,3,4,2015,1,29,8,605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608042,59.76,1998,13,68000,92.9,76.6,95.2,56.8,86.5,81.9,...,0,23,2,2,1,2021,4,16,273,6805
608043,59.76,1998,15,77000,93.2,79.2,97.1,55.0,81.6,81.6,...,0,23,2,2,1,2021,7,31,273,6805
608044,57.66,2003,5,30500,89.0,72.9,96.0,46.6,86.5,77.6,...,0,18,2,2,1,2021,2,27,273,6881
608045,72.22,2003,4,49500,92.9,76.6,95.2,56.8,86.5,81.9,...,0,18,2,2,1,2021,5,4,273,6882


In [6]:
X = df[['전용면적', '건축년도', '층', '소비자태도지수', '현재생활형편지수', '미래생활형편지수',
       '현재경기판단지수', '미래경기판단지수', '내구재구입태도지수', '주택구입태도지수', '현재소비지출지수', '미래소비지출지수',
       '순자산지수', '고용상황지수', '물가예상지수', '코스피지수', '코스닥지수', '코스피200', '코스피50',
       '시장금리', '주택매매가격지수 전국', '주택전세가격지수', '아파트실거래가격지수', '아파트실거래가격지수 전국',
       '생산자물가지수', '소비자물가지수', '주택매매가격지수', '법정동별인구수', '단기 COFIX', '기준금리',
       '총 의사 수', '의과 일반의 인원수', '의과 인턴 인원수', '의과 레지던트 인원수', '의과 전문의 인원수',
       '치과 일반의 인원수', '치과 인턴 인원수', '치과 레지던트 인원수', '치과 전문의 인원수', '한방 일반의 인원수',
       '한방 인턴 인원수', '한방 레지던트 인원수', '한방전문의 인원수', '병원 개수', '보건소 개수', '보건지소 개수',
       '상급종합병원 개수', '요양병원 개수', '의원 개수', '정신병원 개수', '산부인과 개수', '종합병원 개수',
       '치과병원 개수', '치과의원 개수', '한방병원 개수', '한의원 개수', '약국 개수', '역 개수', '공원 면적',
       '대공원 개수', '공원 개수', '한강 근접 여부', '준공 이래 경과년수', '초등학교 개수', '중학교 개수',
       '고등학교 개수', '계약년도', '계약월', '계약일', '법정동', '아파트명']]

y = df['실거래가']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2,
                                                    random_state = 0)

print('Train Data : ', X_train.shape, y_train.shape)
print('Test Data : ', X_test.shape, y_test.shape)

Train Data :  (486437, 71) (486437,)
Test Data :  (121610, 71) (121610,)


### Modeling

### n_estimators = 10, learning_rate = 0.1, max_depth = 4

In [19]:
Model_apt_xgbr = XGBRegressor(n_estimators = 10,
                              learning_rate = 0.1,
                              max_depth = 4,
                              )

In [20]:
%%time

Model_apt_xgbr.fit(X_train, y_train)

CPU times: user 22.5 s, sys: 352 ms, total: 22.8 s
Wall time: 6.17 s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=4,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
# GPU 사용

In [21]:
Model_apt_xgbr = XGBRegressor(n_estimators = 10,
                              learning_rate = 0.1,
                              max_depth = 4,
                              tree_method='gpu_hist')

In [22]:
%%time

Model_apt_xgbr.fit(X_train, y_train)

CPU times: user 1.22 s, sys: 228 ms, total: 1.45 s
Wall time: 1.03 s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=4,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [12]:
y_hat = Model_apt_xgbr.predict(X_test)

In [13]:
mse = mean_squared_error(y_test, y_hat)
np.sqrt(mse)

40368.97826916282

### n_estimators = 500, learning_rate = 0.1, max_depth = 5

In [28]:
%%time
Model_apt_xgbr = XGBRegressor(n_estimators = 500,
                              learning_rate = 0.1,
                              max_depth = 5)
Model_apt_xgbr.fit(X_train, y_train)
y_hat = Model_apt_xgbr.predict(X_test)
mse = mean_squared_error(y_test, y_hat)
np.sqrt(mse)

CPU times: user 20min 13s, sys: 581 ms, total: 20min 14s
Wall time: 5min 5s


In [23]:
np.sqrt(mse)

9697.09196887746

In [ ]:
# gpu

In [24]:
%%time
Model_apt_xgbr = XGBRegressor(n_estimators = 500,
                              learning_rate = 0.1,
                              max_depth = 5,
                              tree_method='gpu_hist')
Model_apt_xgbr.fit(X_train, y_train)
y_hat = Model_apt_xgbr.predict(X_test)
mse = mean_squared_error(y_test, y_hat)
np.sqrt(mse)

CPU times: user 9.82 s, sys: 240 ms, total: 10.1 s
Wall time: 8.32 s


In [25]:
np.sqrt(mse)

10000.41213004584

### n_estimators = 1000, learning_rate = 0.1, max_depth = 10 - 기준

In [7]:
%%time
Model_apt_xgbr = XGBRegressor(n_estimators = 1000,
                              learning_rate = 0.1,
                              max_depth = 10,
                              tree_method='gpu_hist')
Model_apt_xgbr.fit(X_train, y_train)
y_hat = Model_apt_xgbr.predict(X_test)
mse = mean_squared_error(y_test, y_hat)
np.sqrt(mse)

CPU times: user 1min 29s, sys: 910 ms, total: 1min 30s
Wall time: 1min 24s


In [8]:
np.sqrt(mse)

6297.502310542424

### n_estimators = 2000, learning_rate = 0.1, max_depth = 10 (n_estimators 1000 -> 2000)

In [9]:
%%time
Model_apt_xgbr = XGBRegressor(n_estimators = 2000,
                              learning_rate = 0.1,
                              max_depth = 10,
                              tree_method='gpu_hist')
Model_apt_xgbr.fit(X_train, y_train)
y_hat = Model_apt_xgbr.predict(X_test)
mse = mean_squared_error(y_test, y_hat)
np.sqrt(mse)

CPU times: user 2min 58s, sys: 944 ms, total: 2min 59s
Wall time: 2min 45s


In [10]:
np.sqrt(mse)

6235.519174402416

### n_estimators = 1000, learning_rate = 0.05, max_depth = 10 (learning_rate = 0.1 -> 0.05)

In [11]:
%%time
Model_apt_xgbr = XGBRegressor(n_estimators = 1000,
                              learning_rate = 0.05,
                              max_depth = 10,
                              tree_method='gpu_hist')
Model_apt_xgbr.fit(X_train, y_train)
y_hat = Model_apt_xgbr.predict(X_test)
mse = mean_squared_error(y_test, y_hat)
np.sqrt(mse)

CPU times: user 1min 32s, sys: 43.8 ms, total: 1min 32s
Wall time: 1min 24s


In [12]:
np.sqrt(mse)

6444.212521234677

### n_estimators = 1000, learning_rate = 0.2, max_depth = 10 (learning_rate = 0.1 -> 0.2)

In [13]:
%%time
Model_apt_xgbr = XGBRegressor(n_estimators = 1000,
                              learning_rate = 0.2,
                              max_depth = 10,
                              tree_method='gpu_hist')
Model_apt_xgbr.fit(X_train, y_train)
y_hat = Model_apt_xgbr.predict(X_test)
mse = mean_squared_error(y_test, y_hat)
print(np.sqrt(mse))

6459.302354734691
CPU times: user 1min 31s, sys: 36 ms, total: 1min 31s
Wall time: 1min 24s


### n_estimators = 500, learning_rate = 0.1, max_depth = 15 (n_estimators = 1000 -> 500, max_depth = 10 -> 15)

In [7]:
%%time
Model_apt_xgbr = XGBRegressor(n_estimators = 500,
                              learning_rate = 0.1,
                              max_depth = 15,
                              tree_method='gpu_hist')
Model_apt_xgbr.fit(X_train, y_train)
y_hat = Model_apt_xgbr.predict(X_test)
mse = mean_squared_error(y_test, y_hat)
np.sqrt(mse)

CPU times: user 7min 11s, sys: 3.51 s, total: 7min 15s
Wall time: 7min 5s


In [8]:
print(np.sqrt(mse))

7006.923114684135


### n_estimators = 2000, learning_rate = 0.1, max_depth = 15 (n_estimators = 2000, max_depth = 10 -> 15)

In [ ]:
%%time
Model_apt_xgbr = XGBRegressor(n_estimators = 2000,
                              learning_rate = 0.1,
                              max_depth = 15,
                              tree_method='gpu_hist')
Model_apt_xgbr.fit(X_train, y_train)
y_hat = Model_apt_xgbr.predict(X_test)
mse = mean_squared_error(y_test, y_hat)
np.sqrt(mse)

In [ ]:
%%time
Model_apt_xgbr = XGBRegressor(n_estimators = 1000,
                              learning_rate = 0.1,
                              max_depth = ,
                              tree_method='gpu_hist')
Model_apt_xgbr.fit(X_train, y_train)
y_hat = Model_apt_xgbr.predict(X_test)
mse = mean_squared_error(y_test, y_hat)
np.sqrt(mse)

In [ ]:
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=50, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)